# Proyecto Integrador


## Tecnológico de Monterrey
### Maestría en Inteligencia Artificial Aplicada (MNA)
#### Avance 4
#### Equipo 7


* Jorge Arturo Federico Rivera – A01250724
* Marco Antonio Vázquez Morales – A01793704
* Alejandro Jesús Vázquez Navarro - A01793146

## Modelos Alternativos


### Table of Contents
1. [Preparación](##Preparación)
2. [Comparativa](##Comparativa)
3. [Ajuste Fino](##AjusteFino)

Proyecto:

*Modelo clasificador de multimorbilidad maternal y predictor de desenlaces perinatales a partir de datos clínicos metabólicos, genéticos y nutricionales de mujeres mexicanas*

23 de mayo de 2024

#1. [Preparación](##Preparación).

# **1.1 Carga de Librerías**

In [17]:
#%pip install imbalanced-learn
#%pip install scikit-learn tensorflow
#%pip install scikeras
#%pip install --upgrade scikit-learn
#%pip uninstall scikit-learn imbalanced-learn
#%pip install scikit-learn==1.0.2 imbalanced-learn==0.8.1

import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report, precision_recall_curve, auc, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.preprocessing import LabelEncoder


# **1.2. Carga de datos**


In [3]:
file_path = r'data/data_limpia.csv'

#Eliminamos columnas que no necesitaremos como identificarores y el número de clúster

cols_to_remove = ['id_gdg', 'origen_px', 'IndexMorbilidad', 'anticonceptivo_0.0',
       'anticonceptivo_0.6316526610644257', 'anticonceptivo_1.0',
       'anticonceptivo_2.0', 'cluster']

data = pd.read_csv(file_path, sep=";", encoding='utf-8', index_col=False)
data  = data.drop(cols_to_remove , axis=1)

data.head()

,EscalaRiesgo,biopsias,obs_h,ichos_pregesta,hta_pregesta,sop,hipotiroidismo,hipertiroidismo,consumo_alcohol,consumo_tabaco,...,sdg_parto,ant_aborto,macrosomia_rn_0,macrosomia_rn_0.0,macrosomia_rn_0/0,macrosomia_rn_1,macrosomia_rn_1.0,macrosomia_rn_1/0,macrosomia_rn_1/1,macrosomia_rn_2
0,C,0,0,0,0,0,0,0,0,0,...,0.0,-0.747297,0,0,0,1,0,0,0,0
1,C,0,0,0,1,0,0,0,0,0,...,0.0,-0.747297,0,0,0,1,0,0,0,0
2,C,0,1,0,1,0,0,0,0,0,...,0.0,-0.747297,0,0,0,1,0,0,0,0
3,B,0,0,0,1,0,0,0,0,0,...,0.0,-0.747297,0,0,0,1,0,0,0,0
4,C,0,1,0,1,0,0,0,0,0,...,0.0,1.420203,0,0,0,1,0,0,0,0


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

X = data.drop('EscalaRiesgo', axis=1)
y = data['EscalaRiesgo']

# **1.3 Aplicar balanceo de clases via SMOTE**


Esta técnica es recomendada cuando tenemos clases desbalanceadas. Nuestro problema tiene 3 clases:

- A Para alto riesgo (la presencia de 3 o 4 factores multimorbilidad)
- B Para medio riesgo (presencia de 1 o 2 factores multimorbilidad)
- C Para riesgo nulo. (sin la presencia de factores de multimorbilidad)

En temas de salud es importante tener clases balanceadas, esto nos permitirá tener los siguientes beneficios:

- Mejora en la precisión del modelo; en este sentido, detectar una clase **A** con alto riesgo de morbilidad es crucial para que el modelo entregue valor al área médica.

- Equidad en la atención médico. Debemos aseguramos que nuestro modelo sea justo y equitativo sobre todo en temas de salud. Un desbalance puede provocar sesgos y lecturas erróneas.

- Generalización. Un modelo con clases balanceadas generalizará mejor cuando trate nuevos datos. En el dominio de conocimiento de salud humana, esto es relevante.

# 2. [Comparativa](##Comparativa)

## Evaluación con TensorFlow

Es necesario aplicar un LabelEncoder para la variable dependiente ya que es categórica y de tipo string en este momento.

In [9]:
label_encoder = LabelEncoder()

y_encoded = pd.DataFrame(label_encoder.fit_transform(y))

print("y Original\n", y.head(5))
print("y Encoded\n", y_encoded.head(5))

y Original
 0    C
1    C
2    C
3    B
4    C
Name: EscalaRiesgo, dtype: object
y Encoded
    0
0  2
1  2
2  2
3  1
4  2


### Definir CNN

La red neuronal se define con dos capas ocultas, cada una seguida de una Dropout layer para prevenir overfitting, lo cual es clave por la cantidad limitada de datos con los que se cuenta.. Por esta misma última razón, se utiliza regularización L2.

Debido a que la variable dependiente es multi-clase, es necesario agregar una capa de salida de 4 unidades con una activación softmax que es la recomendada al hacer clasificaciones de este tipo.

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],),
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  
    metrics=['accuracy']
)


/home/deck/Documents/School/Proyecto Integrador/Repositorio/MNA_ProyectoIntegrador-Equipo7/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Entrenar modelo y hacer predicciones

In [15]:

history = model.fit(X_train_smote, y_train_smote, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"\nTest accuracy: {test_acc}")

y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)


Epoch 1/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9912 - loss: 0.1061 - val_accuracy: 0.7188 - val_loss: 1.6070
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9859 - loss: 0.1195 - val_accuracy: 0.7143 - val_loss: 1.7468
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9901 - loss: 0.1040 - val_accuracy: 0.7188 - val_loss: 1.6864
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9856 - loss: 0.1209 - val_accuracy: 0.7188 - val_loss: 1.6385
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9880 - loss: 0.1129 - val_accuracy: 0.7188 - val_loss: 1.6140
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9942 - loss: 0.0999 - val_accuracy: 0.7234 - val_loss: 1.6932
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9866 - loss: 0.1198 - val_accuracy: 0.7256 - val_loss: 1.6499
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9929 - loss: 0.0985 - val_accuracy: 0.7143 - val_loss:

### Calcular métricos

In [18]:

recall = recall_score(y_test, y_pred, average='weighted')
print(f"Recall: {recall}")

precision, recall, thresholds = precision_recall_curve(y_test, y_pred_prob[:, 1], pos_label=1)  # Adjust for the class of interest
pr_auc = auc(recall, precision)
print(f"Precision-Recall AUC: {pr_auc}")

f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Score: {f1}")

# Print classification report
print(classification_report(y_test, y_pred))

Recall: 0.9890510948905109
Precision-Recall AUC: 0.968974344837061
F1 Score: 0.9879845694187057
              precision    recall  f1-score   support

           0       0.50      0.33      0.40         3
           1       0.98      0.99      0.98        96
           2       1.00      1.00      1.00       175

    accuracy                           0.99       274
   macro avg       0.83      0.77      0.79       274
weighted avg       0.99      0.99      0.99       274



## Evaluación con Algoritmos de scikit-learn

In [ ]:
def train_and_evaluate_algorithms(X, y):
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Apply SMOTE to balance the classes
    smote = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

    # Check the class distribution
    print("Original class distribution:")
    print(y_train.value_counts())
    print("\nBalanced class distribution:")
    print(y_train_balanced.value_counts())

    # Define the models to be used
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs'),
        'Decision Tree': DecisionTreeClassifier(),
        'Support Vector Classifier': SVC(),
        'K-Nearest Neighbors': KNeighborsClassifier(),
        'Naive Bayes': GaussianNB(),
        'Linear Discriminant Analysis': LinearDiscriminantAnalysis()
    }

    # Initialize a list to store the results
    results = []

    # Train and evaluate each model
    for name, model in models.items():
        start_time = time.time()
        model.fit(X_train_balanced, y_train_balanced)
        training_time = time.time() - start_time

        # Make predictions
        y_pred = model.predict(X_test)

        # Calculate performance metrics
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        # Append the results
        results.append({
            'Model': name,
            'Accuracy': accuracy,
            'F1 Score': f1,
            'Training Time (s)': training_time
        })

    # Convert results to a DataFrame and display
    results_df = pd.DataFrame(results)
    print(results_df)

In [ ]:
target_column = 'EscalaRiesgo'

train_and_evaluate_algorithms(X, y)


Original class distribution:
EscalaRiesgo
C    645
B    305
A      9
Name: count, dtype: int64

Balanced class distribution:
EscalaRiesgo
C    645
B    645
A    645
Name: count, dtype: int64
                          Model  Accuracy  F1 Score  Training Time (s)
0           Logistic Regression  0.963504  0.964074           0.386540
1                 Decision Tree  0.936740  0.935397           0.083869
2     Support Vector Classifier  0.861314  0.856127           0.136673
3           K-Nearest Neighbors  0.664234  0.674319           0.006041
4                   Naive Bayes  0.953771  0.962749           0.017494
5  Linear Discriminant Analysis  0.956204  0.956470           0.067245


# 3. [Ajuste Fino](##AjusteFino)

De acuerdo a la métrica F1 Score, los 2 mejores modelos son:
- Logistic Regression
- Nayve Bayes

Con base en estos dos modelos, realizaremos el afinamiento de hiperparámetros. Definimos una función que realizará el afinamiento de hiperparámetros con base a los 2 modelos elegidos.

In [11]:
def train_and_evaluate_algorithms(X, y):
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Apply SMOTE to balance the classes
    smote = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

    # Check the class distribution
    print("Original class distribution:")
    print(y_train.value_counts())
    print("\nBalanced class distribution:")
    print(y_train_balanced.value_counts())

    # Define the models to be used
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs'),
        'Naive Bayes': GaussianNB(),
    }

    # Initialize a list to store the results
    results = []

    # Train and evaluate each model
    for name, model in models.items():
        start_time = time.time()
        model.fit(X_train_balanced, y_train_balanced)
        training_time = time.time() - start_time

        # Make predictions
        y_pred = model.predict(X_test)

        # Calculate performance metrics
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')

        # Append the results
        results.append({
            'Model': name,
            'Accuracy': accuracy,
            'F1 Score': f1,
            'Training Time (s)': training_time
        })

    # Hyperparameter tuning for Logistic Regression
    log_reg = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
    log_reg_params = {
        'C': [0.1, 1, 10, 100],
        'solver': ['newton-cg', 'lbfgs', 'saga']
    }
    log_reg_grid = GridSearchCV(log_reg, log_reg_params, cv=5, scoring='f1_weighted')
    start_time = time.time()
    log_reg_grid.fit(X_train_balanced, y_train_balanced)
    training_time = time.time() - start_time
    log_reg_best = log_reg_grid.best_estimator_

    # Make predictions
    y_pred = log_reg_best.predict(X_test)

    # Calculate performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Append the results
    results.append({
        'Model': 'Logistic Regression (Tuned)',
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Training Time (s)': training_time
    })

    # Hyperparameter tuning for Naive Bayes (Gaussian)
    naive_bayes = GaussianNB()
    nb_params = {
        'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]
    }
    nb_grid = GridSearchCV(naive_bayes, nb_params, cv=5, scoring='f1_weighted')
    start_time = time.time()
    nb_grid.fit(X_train_balanced, y_train_balanced)
    training_time = time.time() - start_time
    nb_best = nb_grid.best_estimator_

    # Make predictions
    y_pred = nb_best.predict(X_test)

    # Calculate performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Append the results
    results.append({
        'Model': 'Naive Bayes (Tuned)',
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Training Time (s)': training_time
    })

    # Convert results to a DataFrame and display
    results_df = pd.DataFrame(results)
    print(results_df)

In [12]:
train_and_evaluate_algorithms(X, y)


Original class distribution:
EscalaRiesgo
C    645
B    305
A      9
Name: count, dtype: int64

Balanced class distribution:
EscalaRiesgo
C    645
B    645
A    645
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which 

                         Model  Accuracy  F1 Score  Training Time (s)
0          Logistic Regression  0.963504  0.964074           0.363115
1                  Naive Bayes  0.953771  0.962749           0.016687
2  Logistic Regression (Tuned)  0.961071  0.961925         110.308201
3          Naive Bayes (Tuned)  0.953771  0.962749           0.500227
